In [ ]:
# Importar as bibliotecas
import pandas as pd 
import numpy as np

ENDERECO_DADOS = 'https://www.ispdados.rj.gov.br/Arquivos/BaseDPEvolucaoMensalCisp.csv'

# Obtendo dados
try:
    df_roubos = pd.read_csv(ENDERECO_DADOS, sep=';', encoding='iso-8859-1')
    # print(df_roubos['munic'].unique())  # testar as impressões
    print(df_roubos['regiao'].unique())  # testar as impressões
    print(df_roubos.head())
except Exception as e:
    print("Erro ao obter dados do ISP: ", e)


In [ ]:
# Tratando e preparando os dados
try:
    # Delimitando as variáveis
    df_roubos = df_roubos[['cisp', 'regiao', 'munic', 'roubo_veiculo']]
    # print(df_roubos.head())
    # Tratando os erros de acentuação na série região
    correcao_acentuacao = df_roubos['regiao'].str.startswith('Grande Niter', na=False)
    # print(correcao_acentuacao)
    df_roubos.loc[correcao_acentuacao, 'regiao'] = 'Grande Niterói'
    # print(df_roubos['regiao'].unique())

    # Agrupando os dados cisp, região e município
    df_roubos = df_roubos.groupby(['cisp', 'regiao', 'munic']).sum(['roubo_veiculo']).reset_index()

    # Display() é uma função do Jupyter Notebook que permite exibir o DataFrame
    display(df_roubos)
except Exception as e:
    print("Erro no tratamento dos dados: ", e)

In [ ]:
# Calculando as medidas
try:
    array_roubos = np.array(df_roubos['roubo_veiculo'])
    # print(array_roubos)
    
    media = np.mean(array_roubos)
    mediana = np.median(array_roubos)
    total = np.sum(array_roubos)
    # maximo = np.max(array_roubos)
    # minimo = np.min(array_roubos)

    # Obtendo os Quartis
    q1 = np.quantile(array_roubos, 0.25)
    q3 = np.quantile(array_roubos, 0.75)

    # Printando as medidas
    print('Medidas:')
    print(f'Media: {media:.2f}')
    print(f'Mediana: {mediana}')
    print(f'Total: {total}')

except Exception as e:
    print("Erro ao obter as medidas: ", e)

In [ ]:
# Identificando os maiores e menores
try:
    # Gerar um dataframe com os maiores
    # Copy() quando preciar alterar um dataframe já filtrado
    df_maiores =  df_roubos[df_roubos['roubo_veiculo'] > q3].copy()
    df_maiores['flag'] = 'mais'
    # print(df_maiores)

    # Gerar um dataframe com os menores
    df_menores = df_roubos[df_roubos['roubo_veiculo'] < q1].copy()
    df_menores['flag'] = 'menos'
    # print(df_menores)

    # Concatenar os dois dataframes
    df_roubos_flags = pd.concat([df_maiores, df_menores], ignore_index=True)

    display(df_roubos_flags)
except Exception as e:
    print("Erro ao identificar os maiores e menores: ", e)

In [ ]:
# Exportando dados csv ou xlsx
try:
    # Testar sem encoding | Impirmir com o encoding utf-8-sig, se precisar do csv fora do BI
    df_roubos_flags.to_csv('roubos_veiculos.csv', index=False, )

    # df_roubos_flags.to_excel('roubos_veiculos.xlsx', index=False)

except Exception as e:
    print("Erro ao exportar os dados: ", e)
